In [1]:
import numpy as np
import pandas as pd
import segyio
import pickle
from math import floor
from seysmo.data.make_dataset import find_files, calculate_distance, create_input_dict
from matplotlib import pyplot as plt
import plotly.graph_objs as go
import plotly.io as pio
from seysmo.visualization.plotting import plot_map
from seysmo.features.mapping import do_array_for_mapping, compute_y_pred
from seysmo.models.utils import give_data

In [23]:
def create_input_dict_field(filenames, seysm_dict):
    for filename in filenames:
        with segyio.open(filename, "r", endian='big', ignore_geometry=True) as segyfile:
            tr_start = 0
            tr_last = 1
            
            while tr_last != len(segyfile.trace) + 1:
                while segyfile.header[tr_last][segyio.TraceField.FieldRecord] == segyfile.header[tr_start][
                    segyio.TraceField.FieldRecord]:
                    tr_last += 1
                    if tr_last == len(segyfile.trace):
                        break
                seysmogramm = np.array([segyfile.trace[tr_start]])
                for ind in range(tr_start+1, tr_last):
                    seysmogramm = np.concatenate([seysmogramm, np.array([segyfile.trace[ind]])])
                rec_mid_x = floor(
                    ((segyfile.header[tr_start][segyio.TraceField.GroupX] / abs(segyfile.header[0][segyio.TraceField.SourceGroupScalar]) + segyfile.header[tr_last - 1][
                        segyio.TraceField.GroupX] / abs(segyfile.header[0][segyio.TraceField.SourceGroupScalar])) / 2))
                rec_mid_y = floor(
                    ((segyfile.header[tr_start][segyio.TraceField.GroupY] / abs(segyfile.header[0][segyio.TraceField.SourceGroupScalar]) + segyfile.header[tr_last - 1][
                        segyio.TraceField.GroupY] / abs(segyfile.header[0][segyio.TraceField.SourceGroupScalar])) / 2))
                seysm_dict[(rec_mid_x, rec_mid_y, segyfile.header[tr_start][segyio.TraceField.FieldRecord])] = seysmogramm
                tr_start, tr_last = tr_last, tr_last + 1
                print(tr_start, '/', len(segyfile.trace))

In [258]:
seysm_dict = dict()
# create_input_dict_field(["../../data/raw/second_place/Area5_processed_data_for_MASW_ReS.sgy"], seysm_dict)
create_input_dict_field(["D://Vitya/Работа/second_place/Area1_processed_data_for_MASW_ReS.sgy"], seysm_dict)

27 / 189162
54 / 189162
81 / 189162
108 / 189162
135 / 189162
162 / 189162
189 / 189162
216 / 189162
243 / 189162
270 / 189162
297 / 189162
324 / 189162
351 / 189162
378 / 189162
405 / 189162
432 / 189162
459 / 189162
486 / 189162
513 / 189162
540 / 189162
567 / 189162
594 / 189162
621 / 189162
648 / 189162
675 / 189162
702 / 189162
729 / 189162
756 / 189162
783 / 189162
810 / 189162
837 / 189162
864 / 189162
891 / 189162
918 / 189162
945 / 189162
972 / 189162
999 / 189162
1026 / 189162
1053 / 189162
1080 / 189162
1107 / 189162
1134 / 189162
1161 / 189162
1188 / 189162
1215 / 189162
1242 / 189162
1269 / 189162
1296 / 189162
1323 / 189162
1350 / 189162
1377 / 189162
1404 / 189162
1431 / 189162
1458 / 189162
1485 / 189162
1512 / 189162
1539 / 189162
1566 / 189162
1593 / 189162
1620 / 189162
1647 / 189162
1674 / 189162
1701 / 189162
1728 / 189162
1755 / 189162
1782 / 189162
1809 / 189162
1836 / 189162
1863 / 189162
1890 / 189162
1917 / 189162
1944 / 189162
1971 / 189162
1998 / 189162
2025

In [259]:
len(seysm_dict.keys())

7004

In [260]:
zip_data = dict()
# create_input_dict_field(["../../data/raw/second_place/raw_data/Area5_processed_data_for_MASW.sgy"], zip_data)
create_input_dict_field(["D://Vitya/Работа/second_place/raw_data/Area1_processed_data_for_MASW.sgy"], zip_data)

28 / 197217
56 / 197217
84 / 197217
112 / 197217
140 / 197217
168 / 197217
196 / 197217
224 / 197217
252 / 197217
280 / 197217
308 / 197217
336 / 197217
364 / 197217
392 / 197217
420 / 197217
449 / 197217
478 / 197217
506 / 197217
535 / 197217
564 / 197217
592 / 197217
620 / 197217
648 / 197217
677 / 197217
705 / 197217
733 / 197217
761 / 197217
789 / 197217
817 / 197217
845 / 197217
873 / 197217
901 / 197217
929 / 197217
957 / 197217
985 / 197217
1013 / 197217
1041 / 197217
1069 / 197217
1097 / 197217
1125 / 197217
1153 / 197217
1181 / 197217
1209 / 197217
1237 / 197217
1265 / 197217
1293 / 197217
1321 / 197217
1349 / 197217
1377 / 197217
1405 / 197217
1433 / 197217
1461 / 197217
1489 / 197217
1517 / 197217
1545 / 197217
1573 / 197217
1601 / 197217
1629 / 197217
1657 / 197217
1685 / 197217
1713 / 197217
1741 / 197217
1769 / 197217
1797 / 197217
1825 / 197217
1853 / 197217
1881 / 197217
1909 / 197217
1937 / 197217
1965 / 197217
1993 / 197217
2021 / 197217
2049 / 197217
2077 / 197217
21

In [261]:
len(zip_data.keys())

7006

In [262]:
x1, y1, f1 = zip(*list(seysm_dict.keys()))

# Разделяем координаты X и Y для второго массива
x2, y2, f2 = zip(*list(zip_data.keys()))


# Создаем трассировки для каждого массива
trace1 = go.Scatter(
    x=x1,
    y=y1,
    mode='markers',
    marker=dict(color='blue'),
    name='seysmo'
)

trace2 = go.Scatter(
    x=x2,
    y=y2,
    mode='markers',
    marker=dict(color='red'),
    name='ans'
)

# Определяем макет графика
layout = go.Layout(
    title='График массивов с координатами',
    xaxis=dict(title='X'),
    yaxis=dict(title='Y')
)

# Создаем фигуру с данными и макетом
fig = go.Figure(data=[trace1, trace2], layout=layout)

# Отображаем график
pio.show(fig)

In [263]:
big_dict = dict()

In [264]:
x_all, y_all, ffid_all = zip(*list(zip_data.keys()))
all_ffid = set(ffid_all)
for ffid in all_ffid:
    filtered_dict1 = {k: v for k, v in seysm_dict.items() if k[2] == ffid}
    filtered_dict2 = {k: v for k, v in zip_data.items() if k[2] == ffid}
    if len(filtered_dict1) == len(filtered_dict2):
        sorted_filtered_combined1 = sorted(filtered_dict1.items(), key=lambda x: x[0][0])
        sorted_filtered_combined2 = sorted(filtered_dict2.items(), key=lambda x: x[0][0])
        for i in range(len(filtered_dict1)):
            big_dict[sorted_filtered_combined2[i][0]] = sorted_filtered_combined1[i][1]
            del seysm_dict[sorted_filtered_combined1[i][0]]
            del zip_data[sorted_filtered_combined2[i][0]]

In [265]:
print(len(seysm_dict))
print(len(zip_data))

62
64


In [272]:
otv_dict = dict()
otv_dict_x = dict()
outputs_files = find_files("D://Vitya/Работа/second_place/Result_TXT/segy/Area1_results")
for files in outputs_files:    
    with segyio.open(files, "r", endian='big', ignore_geometry=True) as segyfile_output:
        for tr_index in range(len(segyfile_output.header)):
            rec_mid_x = floor(segyfile_output.header[tr_index][segyio.TraceField.CDP_X] / abs(segyfile_output.header[tr_index][segyio.TraceField.SourceGroupScalar]))
            rec_mid_y = floor(segyfile_output.header[tr_index][segyio.TraceField.CDP_Y] / abs(segyfile_output.header[tr_index][segyio.TraceField.SourceGroupScalar]))
            if len(segyfile_output.trace[tr_index]) == 10: 
                if rec_mid_y != 0:
                    otv_dict[(rec_mid_x, rec_mid_y)] = segyfile_output.trace[tr_index]
                else:
                    otv_dict_x[rec_mid_x] = segyfile_output.trace[tr_index]
                    
                    
outputs_files = find_files("D://Vitya/Работа/second_place/Result_TXT/txt/Area1/", '.txt')
def process_group(group):
        # Сортировка по Depth
        sorted_group = group.sort_values(by='Depth')
        # Получаем все значения Velocity в виде списка
        velocity_array = sorted_group['Velocity'].tolist()
        return velocity_array
for files in outputs_files:
    df = pd.read_csv(files, delimiter='\t')
    if 'Receiver Midpoint' not in df.columns:
        df = df[['Receiver_Midpoint_X', 'Depth', 'Velocity']]
        df = df.groupby(['Receiver_Midpoint_X'])
    else:
        df = df[['Receiver Midpoint', 'Depth', 'Velocity']]
        def replace_commas(value):
            if isinstance(value, str):  # Проверяем, является ли значение строкой
                return value.replace(',', '.')
            return value  # Возвращаем значение без изменений, если оно не строка

        df = df.applymap(replace_commas)
        df = df.astype(float)
        df = df.groupby(['Receiver Midpoint'])
    result = df.apply(process_group)
    result_df = pd.DataFrame(result, columns=['Velocity'])
    for mid_x in result_df.index:
        rec_mid_x = floor(mid_x)
        otv_dict_x[rec_mid_x] = result_df.loc[mid_x]

C:\Users\vitya\AppData\Local\Temp\ipykernel_19112\3249497757.py:35: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

C:\Users\vitya\AppData\Local\Temp\ipykernel_19112\3249497757.py:38: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

C:\Users\vitya\AppData\Local\Temp\ipykernel_19112\3249497757.py:35: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

C:\Users\vitya\AppData\Local\Temp\ipykernel_19112\3249497757.py:38: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include

In [273]:
print(len(otv_dict))
print(len(otv_dict_x))
print(len(big_dict))

1774
5289
6942


In [274]:
x1, y1, f1 = zip(*list(big_dict.keys()))

# Разделяем координаты X и Y для второго массива
x2, y2 = zip(*list(otv_dict.keys()))


# Создаем трассировки для каждого массива
trace1 = go.Scatter(
    x=x1,
    y=y1,
    mode='markers',
    marker=dict(color='blue'),
    name='seysmo'
)

trace2 = go.Scatter(
    x=x2,
    y=y2,
    mode='markers',
    marker=dict(color='red'),
    name='ans'
)

# Определяем макет графика
layout = go.Layout(
    title='График массивов с координатами',
    xaxis=dict(title='X'),
    yaxis=dict(title='Y')
)

# Создаем фигуру с данными и макетом
fig = go.Figure(data=[trace1, trace2], layout=layout)

# Отображаем график
pio.show(fig)

In [275]:
last_dict = dict()

In [282]:
x_c, y_c, f_c = zip(*list(big_dict.keys()))
x_o, y_o = zip(*list(otv_dict.keys()))
for i in range(len(x_c)):
    if x_c[i] in x_o and y_c[i] in y_o:
        if x_o.index(x_c[i]) == y_o.index(y_c[i]):
            last_dict[(x_c[i], y_c[i])] = (big_dict[(x_c[i], y_c[i], f_c[i])], otv_dict[(x_c[i], y_c[i])])
            del big_dict[(x_c[i], y_c[i], f_c[i])]
            del otv_dict[(x_c[i], y_c[i])]

In [283]:
len(last_dict)

1727

Предыдущие две ячейки исполнял пока не изменится длина

In [284]:
import plotly.graph_objs as go
import plotly.io as pio
x1, y1, f1 = zip(*list(big_dict.keys()))

# Разделяем координаты X и Y для второго массива
x2, y2 = zip(*list(otv_dict.keys()))


# Создаем трассировки для каждого массива
trace1 = go.Scatter(
    x=x1,
    y=y1,
    mode='markers',
    marker=dict(color='blue'),
    name='seysmo'
)

trace2 = go.Scatter(
    x=x2,
    y=y2,
    mode='markers',
    marker=dict(color='red'),
    name='ans'
)

# Определяем макет графика
layout = go.Layout(
    title='График массивов с координатами',
    xaxis=dict(title='X'),
    yaxis=dict(title='Y')
)

# Создаем фигуру с данными и макетом
fig = go.Figure(data=[trace1, trace2], layout=layout)

# Отображаем график
pio.show(fig)

In [285]:
bad = 0
for_del = []
for x, y, f in big_dict.keys():
    radius = 10
    fl = []

    for (key_x, key_y) in otv_dict.keys():
        distance = calculate_distance(x, y, key_x, key_y)
        if distance <= radius:
            fl.append((distance, key_x, key_y))
    if len(fl) == 0:
        bad += 1
    else:
        dist, x_min, y_min = sorted(fl, key=lambda x: x[0])[0]
        last_dict[(x_min, y_min)] = (
            big_dict[(x, y, f)], otv_dict[(x_min, y_min)])
        for_del.append((x, y, f))
        del otv_dict[(x_min, y_min)]
for i in range(len(for_del)):
    del big_dict[for_del[i]]

In [286]:
print("Записано итого: ", len(last_dict))
print("Осталось ответов c двумя координатами: ", len(otv_dict))
print("Осталось ответов c одной координатой: ", len(otv_dict_x))
print("Осталось сейсмограмм с точными координатами: ", len(big_dict))
print("Осталось сейсмограмм из полных данных: ", len(zip_data))
print("Осталось сейсмограмм из обрезанных данных: ", len(seysm_dict))

Записано итого:  1734
Осталось ответов c двумя координатами:  40
Осталось ответов c одной координатой:  5289
Осталось сейсмограмм с точными координатами:  5208
Осталось сейсмограмм из полных данных:  64
Осталось сейсмограмм из обрезанных данных:  62


In [218]:
import plotly.graph_objs as go
import plotly.io as pio
x1, y1, f1 = zip(*list(big_dict.keys()))

# Разделяем координаты X и Y для второго массива
x2, y2 = zip(*list(otv_dict.keys()))


# Создаем трассировки для каждого массива
trace1 = go.Scatter(
    x=x1,
    y=y1,
    mode='markers',
    marker=dict(color='blue'),
    name='seysmo'
)

trace2 = go.Scatter(
    x=x2,
    y=y2,
    mode='markers',
    marker=dict(color='red'),
    name='ans'
)

# Определяем макет графика
layout = go.Layout(
    title='График массивов с координатами',
    xaxis=dict(title='X'),
    yaxis=dict(title='Y')
)

# Создаем фигуру с данными и макетом
fig = go.Figure(data=[trace1, trace2], layout=layout)

# Отображаем график
pio.show(fig)

Радиус - плохое измерение для нахождение соседей. На карте нам дают искривленный масштаб, точки по y находятся дальше, чем по x. В таких случаях давайте находить ближайшую точку сначала по одной координате, потом по другой.

In [222]:
bad = 0
for_del = []
for x, y, f in big_dict.keys():
    radius = 15
    fl = []

    for (key_x, key_y) in otv_dict.keys():
        distance = calculate_distance(x, 0, key_x, 0)
        if distance <= radius:
            fl.append((key_x, key_y))
    if len(fl) == 0:
        bad += 1
    else:
        x_min, y_min = sorted(fl, key=lambda a: calculate_distance(0, y, 0, a[1]))[0]
        if calculate_distance(x_min, y_min, x, y) < 300:
            last_dict[(x_min, y_min)] = (
                big_dict[(x, y, f)], otv_dict[(x_min, y_min)])
            for_del.append((x, y, f))
            del otv_dict[(x_min, y_min)]
for i in range(len(for_del)):
    del big_dict[for_del[i]]

In [223]:
print("Записано итого: ", len(last_dict))
print("Осталось ответов c двумя координатами: ", len(otv_dict))
print("Осталось ответов c одной координатой: ", len(otv_dict_x))
print("Осталось сейсмограмм с точными координатами: ", len(big_dict))
print("Осталось сейсмограмм из полных данных: ", len(zip_data))
print("Осталось сейсмограмм из обрезанных данных: ", len(seysm_dict))

Записано итого:  33208
Осталось ответов c двумя координатами:  1981
Осталось ответов c одной координатой:  240
Осталось сейсмограмм с точными координатами:  370
Осталось сейсмограмм из полных данных:  0
Осталось сейсмограмм из обрезанных данных:  1976


In [224]:
import plotly.graph_objs as go
import plotly.io as pio
x1, y1, f1 = zip(*list(big_dict.keys()))

# Разделяем координаты X и Y для второго массива
x2, y2 = zip(*list(otv_dict.keys()))


# Создаем трассировки для каждого массива
trace1 = go.Scatter(
    x=x1,
    y=y1,
    mode='markers',
    marker=dict(color='blue'),
    name='seysmo'
)

trace2 = go.Scatter(
    x=x2,
    y=y2,
    mode='markers',
    marker=dict(color='red'),
    name='ans'
)

# Определяем макет графика
layout = go.Layout(
    title='График массивов с координатами',
    xaxis=dict(title='X'),
    yaxis=dict(title='Y')
)

# Создаем фигуру с данными и макетом
fig = go.Figure(data=[trace1, trace2], layout=layout)

# Отображаем график
pio.show(fig)

In [225]:
x1, y1, f1 = zip(*list(seysm_dict.keys()))

# Разделяем координаты X и Y для второго массива
x2, y2 = zip(*list(otv_dict.keys()))


# Создаем трассировки для каждого массива
trace1 = go.Scatter(
    x=x1,
    y=y1,
    mode='markers',
    marker=dict(color='blue'),
    name='seysmo'
)

trace2 = go.Scatter(
    x=x2,
    y=y2,
    mode='markers',
    marker=dict(color='red'),
    name='ans'
)

# Определяем макет графика
layout = go.Layout(
    title='График массивов с координатами',
    xaxis=dict(title='X'),
    yaxis=dict(title='Y')
)

# Создаем фигуру с данными и макетом
fig = go.Figure(data=[trace1, trace2], layout=layout)

# Отображаем график
pio.show(fig)

Видим много разниц по X координате, малую по Y. Это позволяет попробовать применить поиск по радиусу на минимальном расстоянии

In [226]:
bad = 0
for_del = []
for x, y, f in seysm_dict.keys():
    radius = 15
    fl = []

    for (key_x, key_y) in otv_dict.keys():
        distance = calculate_distance(x-10, 0, key_x, 0)
        if distance <= radius:
            fl.append((key_x, key_y))
    if len(fl) == 0:
        bad += 1
    else:
        x_min, y_min = sorted(fl, key=lambda a: calculate_distance(0, y, 0, a[1]))[0]
        if calculate_distance(x_min, y_min, x, y) < 40:
            last_dict[(x_min, y_min)] = (
                seysm_dict[(x, y, f)], otv_dict[(x_min, y_min)])
            for_del.append((x, y, f))
            del otv_dict[(x_min, y_min)]
for i in range(len(for_del)):
    del seysm_dict[for_del[i]]

In [227]:
print("Записано итого: ", len(last_dict))
print("Осталось ответов c двумя координатами: ", len(otv_dict))
print("Осталось ответов c одной координатой: ", len(otv_dict_x))
print("Осталось сейсмограмм с точными координатами: ", len(big_dict))
print("Осталось сейсмограмм из полных данных: ", len(zip_data))
print("Осталось сейсмограмм из обрезанных данных: ", len(seysm_dict))

Записано итого:  35022
Осталось ответов c двумя координатами:  167
Осталось ответов c одной координатой:  240
Осталось сейсмограмм с точными координатами:  370
Осталось сейсмограмм из полных данных:  0
Осталось сейсмограмм из обрезанных данных:  162


In [249]:
x1, y1, f1 = zip(*list(seysm_dict.keys()))

# Разделяем координаты X и Y для второго массива
x2, y2 = zip(*list(otv_dict.keys()))


# Создаем трассировки для каждого массива
trace1 = go.Scatter(
    x=x1,
    y=y1,
    mode='markers',
    marker=dict(color='blue'),
    name='seysmo'
)

trace2 = go.Scatter(
    x=x2,
    y=y2,
    mode='markers',
    marker=dict(color='red'),
    name='ans'
)

# Определяем макет графика
layout = go.Layout(
    title='График массивов с координатами',
    xaxis=dict(title='X'),
    yaxis=dict(title='Y')
)

# Создаем фигуру с данными и макетом
fig = go.Figure(data=[trace1, trace2], layout=layout)

# Отображаем график
pio.show(fig)

ValueError: not enough values to unpack (expected 3, got 0)

In [247]:
bad = 0
for_del = []
for x, y, f in seysm_dict.keys():
    radius = 30
    fl = []

    for (key_x, key_y) in otv_dict.keys():
        distance = calculate_distance(x-10, 0, key_x, 0)
        if distance <= radius:
            fl.append((key_x, key_y))
    if len(fl) == 0:
        bad += 1
    else:
        x_min, y_min = sorted(fl, key=lambda a: calculate_distance(0, y, 0, a[1]))[0]
        if calculate_distance(x_min, y_min, x-20, y) < 240:
            last_dict[(x_min, y_min)] = (
                seysm_dict[(x, y, f)], otv_dict[(x_min, y_min)])
            for_del.append((x, y, f))
            del otv_dict[(x_min, y_min)]
for i in range(len(for_del)):
    del seysm_dict[for_del[i]]

In [248]:
print("Записано итого: ", len(last_dict))
print("Осталось ответов c двумя координатами: ", len(otv_dict))
print("Осталось ответов c одной координатой: ", len(otv_dict_x))
print("Осталось сейсмограмм с точными координатами: ", len(big_dict))
print("Осталось сейсмограмм из полных данных: ", len(zip_data))
print("Осталось сейсмограмм из обрезанных данных: ", len(seysm_dict))

Записано итого:  35184
Осталось ответов c двумя координатами:  5
Осталось ответов c одной координатой:  240
Осталось сейсмограмм с точными координатами:  370
Осталось сейсмограмм из полных данных:  0
Осталось сейсмограмм из обрезанных данных:  0


Постепенно увеличивал радиус для y, вычитаю координату так как на графике вижу куда смещение идет. Возможны неточности, но как есть.

In [287]:
x1, y1, f1 = zip(*list(big_dict.keys()))

# Разделяем координаты X и Y для второго массива
x2 = list(otv_dict_x.keys())


# Создаем трассировки для каждого массива
trace1 = go.Scatter(
    x=x1,
    y=[0]*len(x1),
    mode='markers',
    marker=dict(color='blue'),
    name='seysmo'
)

trace2 = go.Scatter(
    x=x2,
    y=[0]*len(x2),
    mode='markers',
    marker=dict(color='red'),
    name='ans'
)

# Определяем макет графика
layout = go.Layout(
    title='График массивов с координатами',
    xaxis=dict(title='X'),
    yaxis=dict(title='Y')
)

# Создаем фигуру с данными и макетом
fig = go.Figure(data=[trace1, trace2], layout=layout)

# Отображаем график
pio.show(fig)

Решено не добавлять оставшиеся точки из-за возможных ошибок. 200 сейсмограмм на выборку сильно не повлияют.

In [288]:
bad = 0
for_del = []
for x, y, f in big_dict.keys():
    radius = 3
    fl = []

    for key_x in otv_dict_x.keys():
        distance = calculate_distance(x, 0, key_x, 0)
        if distance <= radius:
            fl.append(key_x)
    if len(fl) == 0:
        bad += 1
    else:
        x_min = sorted(fl, key=lambda a: calculate_distance(0, x, 0, a))[0]
        last_dict[(x_min, y)] = (
            big_dict[(x, y, f)], otv_dict_x[x_min])
        for_del.append((x, y, f))
        del otv_dict_x[x_min]
for i in range(len(for_del)):
    del big_dict[for_del[i]]

In [289]:
print("Записано итого: ", len(last_dict))
print("Осталось ответов c двумя координатами: ", len(otv_dict))
print("Осталось ответов c одной координатой: ", len(otv_dict_x))
print("Осталось сейсмограмм с точными координатами: ", len(big_dict))
print("Осталось сейсмограмм из полных данных: ", len(zip_data))
print("Осталось сейсмограмм из обрезанных данных: ", len(seysm_dict))

Записано итого:  6123
Осталось ответов c двумя координатами:  40
Осталось ответов c одной координатой:  899
Осталось сейсмограмм с точными координатами:  818
Осталось сейсмограмм из полных данных:  64
Осталось сейсмограмм из обрезанных данных:  62


In [252]:
x1, y1, f1 = zip(*list(big_dict.keys()))

# Разделяем координаты X и Y для второго массива
x2 = list(otv_dict_x.keys())


# Создаем трассировки для каждого массива
trace1 = go.Scatter(
    x=x1,
    y=[0]*len(x1),
    mode='markers',
    marker=dict(color='blue'),
    name='seysmo'
)

trace2 = go.Scatter(
    x=x2,
    y=[0]*len(x2),
    mode='markers',
    marker=dict(color='red'),
    name='ans'
)

# Определяем макет графика
layout = go.Layout(
    title='График массивов с координатами',
    xaxis=dict(title='X'),
    yaxis=dict(title='Y')
)

# Создаем фигуру с данными и макетом
fig = go.Figure(data=[trace1, trace2], layout=layout)

# Отображаем график
pio.show(fig)

Получилось 35184 пары, что считаю успехом

In [290]:
final_dict = dict()
b = 0
for key in last_dict.keys():
    seysmogramm, otv = last_dict[key]
    if otv.shape == (10,) and seysmogramm.shape == (27, 313):
        final_dict[key] = (seysmogramm, otv)
    else:
        if otv.shape == (1,):
            if np.array(otv[0]).shape == (10, ):
                final_dict[key] = (seysmogramm, np.array(otv[0]))
            else:
                b += 1
        else:
            b += 1
print("Неудачных совпадений ", b, " из ", len(last_dict))

Неудачных совпадений  0  из  6123


C:\Users\vitya\AppData\Local\Temp\ipykernel_19112\3766298926.py:9: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\vitya\AppData\Local\Temp\ipykernel_19112\3766298926.py:10: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [291]:
with open(f'../../data/processed/Area1.pkl', 'wb') as f:
    pickle.dump(final_dict, f)

Area1: ~900 недобавленных сейсмограмм
Area2: 10 недобавленных сейсмограмм
Area34: ~200 недобавленных сейсмограмм
Area5: ~300 недобавленных сейсмограмм
Area6: ~400 недобавленных сейсмограмм

In [292]:
with open(f'../../data/processed/Area1.pkl', 'rb') as f:
    ar1 = pickle.load(f)
with open(f'../../data/processed/Area2.pkl', 'rb') as f:
    ar2 = pickle.load(f)
with open(f'../../data/processed/Area34.pkl', 'rb') as f:
    ar34 = pickle.load(f)
with open(f'../../data/processed/Area5.pkl', 'rb') as f:
    ar5 = pickle.load(f)
with open(f'../../data/processed/Area6.pkl', 'rb') as f:
    ar6 = pickle.load(f)
    
    
result = ar1 | ar2 | ar34 | ar5 | ar6

In [293]:
len(result)

52620

In [294]:
with open(f'../../data/processed/all_data.pkl', 'wb') as f:
    pickle.dump(result, f)